In [ ]:
#allows imports from other folders in project
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# NBEATS MODEL

## Load Data and Create Model

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from Models.NBEATS import NBeats
from dataPreprocessing import load_data, prepare_dataloader


# Load Data and Create Model
train_file = "../Data/Train/train1990s.csv"
config = {
    "use_fft": False,  # No FFT needed
    "use_exog": False  # No exogenous variables
}
X_train, y_train, X_valid, y_valid, X_test, y_test, dates, y_scaler = load_data(train_file, config=config)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = NBeats(input_size=X_train.shape[1]).to(device)


## Train

In [ ]:
#train with extant module:
from PyTorchModular import train_model

# Prepare DataLoaders
dataloader_train = prepare_dataloader(X_train, y_train)
dataloader_valid = prepare_dataloader(X_valid, y_valid)
dataloader_test = prepare_dataloader(X_test, y_test)

# Training Setup
loss_fn = nn.MSELoss()
optimizer = optim.AdamW(model.parameters(), lr=0.005, weight_decay=5e-7)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)

savepath = '../Models/Weights/'
trainingMetadata = train_model(model, 120, savepath, 'nbeats', dataloader_train, dataloader_valid, loss_fn, optimizer, device)

## Visualise Output

In [ ]:
from Evaluation.evaluation_helpers import evaluate_model, get_best_path
plt.ioff() #use this if you want to control when the visualisation is shown

#find the path to the best model of this kind (may not be the model trained above if training has been run several times)
bestPath = get_best_path(savepath, model.__class__.__name__.lower())
#make predictions using this model
predictions = evaluate_model(model, bestPath, X_test, device, y_scaler)
# Convert y_test Back to Original Scale
y_test_original = y_scaler.inverse_transform(y_test)

# Final Improved Graph
plt.figure(figsize=(16, 8))
plt.plot(dates[-len(y_test_original):], y_test_original, label="Actual PCEPI", color="blue", linewidth=2)
plt.plot(dates[-len(predictions):], predictions, label="Predicted PCEPI", linestyle="dashed", color="orange", linewidth=2)
plt.xlabel("Date")
plt.ylabel("PCEPI")
plt.title("Final Optimized N-BEATS PCE Prediction (Residual Learning + Fourier Features)")
plt.legend()
plt.grid()
plt.show()

## Evaluation

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
# Compute MAE
mae = mean_absolute_error(y_test_original, predictions)

# Compute RMSE
rmse = np.sqrt(mean_squared_error(y_test_original, predictions))

print(f"Mean Absolute Error (MAE): {mae:.5f}")
print(f"Root Mean Square Error (RMSE): {rmse:.5f}")

In [ ]:
# Compute residual errors
residuals = y_test_original - predictions

# Visualise residual errors
plt.figure(figsize=(16, 8))
plt.plot(dates[-len(residuals):], residuals, label="Residual Errors", color="red", linewidth=2)
plt.axhline(y=0, color='black', linestyle='dashed')  # Reference line at 0
plt.xlabel("Date")
plt.ylabel("Residual Error")
plt.title("Residual Errors Over Time")
plt.legend()
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(residuals, bins=30, edgecolor='black', color='red', alpha=0.7)
plt.xlabel("Residual Error")
plt.ylabel("Frequency")
plt.title("Histogram of Residuals")
plt.axvline(x=0, color='black', linestyle='dashed')  # Reference line at 0
plt.grid()
plt.show()